In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [16]:
# =========================================================
# CONFIG
# =========================================================
DATASET_PATH = "C:/Users/USER/Desktop/FCI/8vo Ciclo/Inteligencia Artificial Aplicada/dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123


In [17]:
# =========================================================
# 1. CARGA DEL DATASET USANDO 70/15/15
# =========================================================

# 70% → train, 30% → temp
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.30,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# 30% temporal → luego se divide en val y test
temp_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.30,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Dividir temp_ds en validation y test
temp_ds = temp_ds.cache()
val_size = len(temp_ds) // 2

val_ds = temp_ds.take(val_size)
test_ds = temp_ds.skip(val_size)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Clases detectadas:", class_names)
print("Total clases:", num_classes)

Found 32517 files belonging to 12 classes.
Using 22762 files for training.
Found 32517 files belonging to 12 classes.
Using 9755 files for validation.
Clases detectadas: ['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'dataset', 'healthy', 'powdery_mildew']
Total clases: 12


In [19]:
# =========================================================
# 3. MODELO RESNET (TRANSFER LEARNING) — SIN DATA AUGMENTATION
# =========================================================

base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)

base_model.trainable = False   # Etapa 1: congelado

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))

# Pre-procesamiento oficial de ResNet
x = tf.keras.applications.resnet50.preprocess_input(inputs)

# Extracción de características
x = base_model(x, training=False)

# Clasificador denso
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)


In [ ]:
# =========================================================
# 4. ENTRENAMIENTO: ETAPA 1 (BASE CONGELADA)
# =========================================================
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n======= ENTRENANDO ETAPA 1 (BASE CONGELADA) =======")
model.fit(train_ds, validation_data=val_ds, epochs=10)



======= ENTRENANDO ETAPA 1 (BASE CONGELADA) =======
Epoch 1/10
712/712 ━━━━━━━━━━━━━━━━━━━━ 1182s 2s/step - accuracy: 0.7256 - loss: 0.8343 - val_accuracy: 0.8699 - val_loss: 0.4374
Epoch 2/10
 68/712 ━━━━━━━━━━━━━━━━━━━━ 17:04 2s/step - accuracy: 0.8287 - loss: 0.5118

In [ ]:
# =========================================================
# 5. ETAPA 2: FINE-TUNING (DESCONGELAR ÚLTIMAS CAPAS)
# =========================================================
for layer in base_model.layers[-3:]:  # últimas ~3 capas
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n======= ENTRENANDO ETAPA 2 (FINE-TUNING) =======")
model.fit(train_ds, validation_data=val_ds, epochs=10)


In [ ]:
# =========================================================
# 6. PREDICCIONES PARA MÉTRICAS
# =========================================================
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

y_true = np.array(y_true)
y_pred = np.array(y_pred)



In [ ]:
# =========================================================
# 7. MÉTRICAS: ACCURACY, PRECISION, RECALL, F1
# =========================================================
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average="weighted")
rec = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

print("\n======= MÉTRICAS DEL MODELO =======")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

print("\n======= CLASSIFICATION REPORT =======")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# =========================================================
# 8. MATRIZ DE CONFUSIÓN
# =========================================================
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names
)
plt.xlabel("Predicción")
plt.ylabel("Etiqueta real")
plt.title("Matriz de Confusión")
plt.show()


In [ ]:
# =========================================================
# 9. GUARDAR MODELO
# =========================================================
model.save("resnet_tomato_11clases_finetuned.h5")
print("\nModelo guardado como resnet_tomato_11clases_finetuned.h5")